# Sommes distribuées de matrices avec Dask

Ce notebook montre plusieurs choses :
  * Comment démarrer un cluster Dask depuis un notebook sur TREX
  * Comment utiliser Dask pour paralléliser des appels de fonctions avec *dask.delayed*

## Create cluster

Nous allons démarrer un cluster Dask. Ce cluster sera un Slurm cluster qui se connectera au cluster TREX.

### Imports
Pour créer un cluster SLURM, nous allons utiliser le module *dask_jobqueue* . Il permet d'initialiser un cluster en quelques lignes  depuis un notebook.

In [1]:
from dask_jobqueue import SLURMCluster
from distributed import LocalCluster, Client

### Cluster initialisation

Le cluster sera composé de workers Dask, lancé via des jobs SLURM. Chaque job SLURM lancera 4 worker chacun utilisant 2 cpus et 16 GB de mémoire dans SLURM. 

In [ ]:
account_Trex = 'formation_isae'
partition_Trex = "cpu19_rh8"
qos_Trex = "--qos=cpu_2019_40"

cluster = SLURMCluster(
    # Dask-worker specific keywords
    n_workers=4,  # start 4 workers
    cores=2,  # each worker runs on 2 cores
    memory="16GB",  # each worker uses 16GB memory
    processes=1,  # Number of Python processes to cut up each job
    local_directory="$TMPDIR",  # Location to put temporary data if necessary
    account=account_Trex,
    queue=partition_Trex,
    walltime="01:00:00",
    interface="ib0",
    log_directory="../dask-logs",
    job_extra_directives=[qos_Trex] # qos to use
)
cluster

Pour le moment, il n'y a pas de worker Dask. Il est possible d'instancier le client pour voir les workers

In [ ]:
client = Client(cluster)
client

Nous pouvons voir que nous avons dorénavant des workers Dask.

## Compute facets contribution
Nous allons maintenant appliquer des sommes distribuées de matrices avec le cluster que nous venons de créer. Pour cela, nous aurons besoin de *dask.delayed*, qui permet d'indiquer à Dask la fonction que nous souhaitons paralléliser.

In [4]:
import dask.delayed
import dask.array as da
import numpy as np

In [5]:
# List of MNT facets
facets = range(10)

Ici, nous définissons la fonction qui sera parallélisée.

In [6]:
# Fonction that takes a facet and returns a matrix corresponding
# to the controibution of the facet for the image
def my_f(i):
    return i * np.ones((2048 * 8, 2048 * 8), dtype=int)

Et ensuite, nous définissons un vecteur de fonction *delayed* pour Dask. Lorsque nous utilisons *delayed*, la fonction n'est pas exécutée directement. A la place, Dask en fait un *delayed object* qui permet de tracer les fonctions à exécuter et ses arguments (ici, chaque valeur du vecteur facets).  

In [ ]:
lazy_arrays =  [dask.delayed(my_f)(i) for i in facets]
lazy_arrays

Nous pouvons voir les fonctions *my_f* dans les objets *Delayed*. Comme nous avions 10 éléments dans *facets*, nous avons donc dix fonctions.  
A partir de ces fonctions dites *lazy* (elles ne s'exécutent pas directement), nous indiquons à Daks d'en faire des vecteurs avec l'abstration mémoire des dask array. Une fonction from_delayed existe pour transformer un delayed en arry. Ensuite, nous utilisons *stack* afin d'obtenir un bloc (https://docs.dask.org/en/stable/generated/dask.array.stack.html).

In [8]:
arrays = [
    da.from_delayed(
        lazy_array, dtype=int, shape=(2048 * 8, 2048 * 8)  # for every lazy value
    )
    for lazy_array in lazy_arrays
]

stack = da.stack(arrays, axis=0)
stack

dask.array<stack, shape=(10, 16384, 16384), dtype=int64, chunksize=(1, 16384, 16384), chunktype=numpy.ndarray>

Dask propose une visualisation dans Jupyter qui permet d'avoir une vision du bloc de données. Si nous décidons de redéfinir la taille d'un chunk dans le bloc, Dask se met à jour : 

In [9]:
stack = stack.rechunk((1, 4096, 4096))
stack

dask.array<rechunk-merge, shape=(10, 16384, 16384), dtype=int64, chunksize=(1, 4096, 4096), chunktype=numpy.ndarray>

In [10]:
type(stack)

dask.array.core.Array

Maintenant, l'objectif est de faire la somme sur l'axe 0 (sur les 10 facets déclarées). La transformation sum est directement accessible (https://docs.dask.org/en/stable/generated/dask.array.sum.html).

In [11]:
stack_sum = stack.sum(axis=0)
stack_sum

dask.array<sum-aggregate, shape=(16384, 16384), dtype=int64, chunksize=(4096, 4096), chunktype=numpy.ndarray>

L'ojectif est de "faire partir" le calcul sur l'axe Y du premier élement (index 0). Le résultat est le même sur l'ensemble des éléments [0, 16383]

In [13]:
result = stack_sum[0, :].compute()
result

array([45, 45, 45, ..., 45, 45, 45])

In [14]:
type(result)

numpy.ndarray

In [15]:
client.close()
cluster.close()